In [1]:
import tensorflow as tf
import wandb
import audiomentations
from pathlib import Path
import recorder
from scipy.io.wavfile import read
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import time
import struct
import wave
from pvrecorder import PvRecorder
import IPython
import os
from utils.data_processing import augmenter,log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect


In [2]:
os.environ["WANDB_SILENT"] = "false"

In [3]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
cm.update(
    "rise",
    {
        "theme": "sky",
        "transition": "zoom",
        "start_slideshow_at": "selected",
        'scroll':'false'
     }
)

{'theme': 'sky',
 'transition': 'zoom',
 'start_slideshow_at': 'selected',
 'scroll': 'false'}

![](slides/tiny_ml_cover.svg)

![](slides/ml_morphology.svg)

![](slides/mlops_graph.svg)

![](slides/edge_devices.svg)

<!DOCTYPE html>
<html>
<table style="height:1000px;width:100%">
 <tr>
    <td> <img src="./slides/spark_micro_mod_pico_1.jpg" alt="Drawing" style="height: 525px;width:800px;" align="right"/> </td>
    <td> <img src="./slides/micro_mod_pico_2.jpg" alt="Drawing" style="height: 850px; width:900px" align="left"/> </td>
    </tr>
</table>

In [4]:
tqdm_args = dict(leave=True,position=0)
tqdm_colours = (f'#{r:02x}{g:02x}{b:02x}'for r,g,b in [(int(255*s), 3, int(255*s)) for s in np.linspace(1,0,100)])
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='Data')

wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin


In [5]:
recorder = PvRecorder(device_index=0, frame_length=512)
sound = []
recorder.start()
print('recording')
t_0 = time.time()
while time.time()-t_0<4:
    frame = recorder.read()
    sound.extend(frame)
recorder.stop()

recording


In [6]:
data = np.array(sound).astype(np.int16)
t_step = np.arange(0, len(data))
fig = px.line(x=t_step, y=data, title='a sound wave')
run.log({'sound_wave': fig})

### Showing a Run in Notebook

In [7]:
run

In [8]:
path = './data/yes/yes_record 102.wav'
run.log({'test_sound':wandb.Audio(path)})
run

# The basis of our signal processing is using Fast Fourier Transform (FFT)
> - [here](https://www.youtube.com/watch?v=spUNpyF58BY) is an amazing video on how FFT works. 
> - quickly go through getting from FFT to a Spectrogram. 
>  ![](./slides/200px-Fourier_in_his_coat_of_prefect.jpg)

In [9]:
fft = np.fft.fft(data).astype(np.int16)
t_steps = np.arange(0,len(fft))
fft = np.stack([fft,t_step]).T
df = pd.DataFrame(fft, columns = ['Faurrier_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Faurrier_Value', title='Fourrier Transform')
run.log({'Fouriere Transform':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/660930182.py:1: ComplexWarning:

Casting complex values to real discards the imaginary part

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interf

In [10]:
run

## FFT can also be inverted, we can get sound back

In [11]:
fft = np.fft.fft(data)
roll = np.roll(fft,200)
ifft = np.fft.ifft(roll).astype(np.int16)
t_step = np.arange(len(ifft))
ifft = np.stack([ifft,t_step]).T
df = pd.DataFrame(ifft, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', 
              y='Frequecy_Value', 
              title='Inverse Fourrier Transform to Get sound Back')
run.log({'Inverset Fouriere':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/1994990191.py:3: ComplexWarning:

Casting complex values to real discards the imaginary part



In [12]:
run

## Down Sample using FFT as an example of Signal Processing

In [13]:
# and examper of downsampling using FFT
roll = np.roll(fft,15)
ifft_ten = np.fft.ifft(roll)
ifft_ten = ((2**(16-4)) * ifft_ten/ifft_ten.max()).astype(np.int16)
ifft_ten = np.stack([ifft_ten,t_step]).T
df = pd.DataFrame(ifft_ten, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Frequecy_Value', title='Dowsampling using Fast Fourrier Transform (FFT)')
run.log({'Down_Sampled_Sound':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/3482508054.py:4: ComplexWarning:

Casting complex values to real discards the imaginary part



In [14]:
run

In [15]:
run.finish()
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

### ARM spectrogram using FFT 🦾


In [16]:
category = 'yes'
idx = 10
sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
sound = sound.astype(np.float32, order='F') / 32768.0
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
get_arm_spectrogram(sound)

array([[0.5     , 0.3125  , 0.125   , ..., 0.      , 0.      , 0.      ],
       [0.671875, 0.375   , 0.0625  , ..., 0.015625, 0.      , 0.      ],
       [0.78125 , 0.390625, 0.03125 , ..., 0.      , 0.      , 0.      ],
       ...,
       [1.109375, 0.53125 , 0.125   , ..., 0.015625, 0.      , 0.      ],
       [0.96875 , 0.375   , 0.03125 , ..., 0.015625, 0.      , 0.      ],
       [0.921875, 0.34375 , 0.0625  , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

### Showing output of custom function optimized for arm processor

In [17]:
category = 'yes'
sound = read_wav(f'./data/{category}/yes_record 119.wav')
print(sound)
sound = sound.astype(np.float32, order='F') / 32768.0
# what we are using 
arm_spct = get_arm_spectrogram(sound)
# what were not using 
tf_spect = tf.signal.stft(sound, frame_length=512, frame_step=128).numpy().astype(np.float32)
fig = px.imshow(np.array(arm_spct))
run.log({f'spectrogram_{category}':fig})

[   0    0    0 ... -591 -643 -631]


/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/3992365391.py:8: ComplexWarning:

Casting complex values to real discards the imaginary part



In [18]:
run 

# Adding spectrogram plot to wandb table 🌌

# logging some media outside of a table to workspace
This is for:
- cheching what our network see in signal domain;
- to spot patterns;
- gain intuitions about process and raw signal data.

In [19]:
sample_rate = 16000
category = 'yes'
n_samples = 10
for idx in tqdm(range(1,n_samples), colour=next(tqdm_colours)):
    sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
    sound = sound.astype(np.float32, order='F') / 32768.0
    arm_spect = get_arm_spectrogram(sound)
    img = plot_spectrogram(arm_spect,sample_rate=sample_rate)
    img = wandb.Image(img)
    run.log({f'spectrogram_image{category}':img})

100%|█████████████████████████████████████████████| 9/9 [00:02<00:00,  3.02it/s]


In [20]:
run

In [21]:
#  !mkdir datasets
#  !wget 'https://github.com/karoldvl/ESC-50/archive/master.zip' -P ~/pico/audio-classifier/datasets
#  !unzip -q ~/datasets/master.zip -d ./datasets/

# This Part covers pre training our model on ESC 50 dataset

### Reading our pre-traning data to pandas

In [25]:
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection', 
                 group='Data',
                 resume='must')
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
df = pd.read_csv(esc50_csv)
initial_data_table = wandb.Table(data=df, columns=list(df.columns))
run.log({'initial_table':initial_data_table})

# building a wandb Artifact for the generic dataset for pre training

In [26]:
from pathlib import Path

artifact = wandb.Artifact(type='sound_data',
                          name='ESC-50-master')
artifact.add_dir('./datasets/')
run.log_artifact(artifact)

wandb: Adding directory to artifact (./datasets)... Done. 1.9s


### Here we are training on 1 second sound clips and our ESC 50 Dataset is 4 seconds per class

### testing our function 

In [28]:
test_file = 'datasets/ESC-50-master/audio/1-100032-A-0.wav'
data = segment(fid=test_file,chunk=1)

### reading 1 second clips writing to a new files

In [69]:
in_paranet = Path('./datasets/ESC-50-master/audio/')
out_data_dir = Path('ESC-50')
out_data_dir.mkdir(exist_ok=True)
all_out = [ ]
for idx, row in tqdm(df.iterrows()):
    out_dir = out_data_dir/row.category
    out_dir.mkdir(exist_ok=True)
    data, vals = segment(fid = str(in_paranet/row.filename),
       chunk=1)
    chans, samp_width , rate = vals
    out_fids = [ ]
    for idx,sound in enumerate(data):
        out_fid = out_dir/f'{idx}_{row.filename}'
        out_fids.append(out_fid)
        with wave.open(str(out_fid), 'w') as outfile:
            outfile.setnchannels(chans)
            outfile.setsampwidth(samp_width)
            outfile.setframerate(rate)
            outfile.setnframes(int(len(sound) /  samp_width))
            outfile.writeframes(sound)
    all_out.append(out_fids)


12000it [00:11, 1021.44it/s]


# Building an Artifact of 1 second sound clips broken down into class/category name
This:
> - creates a type which is here a parent directory
> - names an artfact for each class
> - creates a list of artfacts which are then itterated through to add a dirctory

#### Our artifacts are going to follow this pattern

In [71]:
import shutil
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

esc50_csv = Path('./datasets/ESC-50-master/meta/esc50.csv')
path = Path('ESC-50/')
meta_data = path/'meta_data'
meta_data.mkdir(exist_ok=True)
shutil.copyfile(esc50_csv,meta_data/esc50_csv.name)
esc_artifacts = [pth for pth in path.iterdir()]
for path in tqdm(esc_artifacts[-15:],**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)
    
run.finish()


100%|███████████████████████████████████████████| 15/15 [00:04<00:00,  3.28it/s]


wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.


### A Function to visualize our spectrogram: 

In [75]:
def plot_spectrogram(spectrogram,sample_rate, vmax=None):
    '''a func to computer spect and save as a png then read as np.array'''
    plt.ioff()
    transposed_spectrogram = tf.transpose(spectrogram)
    height = transposed_spectrogram.shape[0]
    X = np.arange(transposed_spectrogram.shape[1])
    Y = np.arange(height * int(sample_rate / 256), step=int(sample_rate / 256))

    fig, ax = plt.subplots(1,1)
    ax.pcolormesh(X, Y, tf.transpose(spectrogram), vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    
    fig.tight_layout()
    plt.savefig('img.jpg',dpi=300)
    plt.close()
    return cv2.imread('img.jpg')

### adding our one second sound file paths to our data frame

In [76]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
df = pd.read_csv(esc50_csv)
esc50_csv = meta_data/esc50_csv.name
sec_files = np.array(all_out).astype(str)
for files in range(sec_files.shape[-1]):
    df[f'{files}_{files+1}_sec']=sec_files[...,files].astype(str)
df.to_csv(esc50_csv)
log_wandb_artifact(run=run,path=esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [77]:
run

In [78]:
meta_data_table = wandb.Table(data=df,columns=list(df.columns))
run.log({'meta_data_without_media':meta_data_table})


# Data table for sound Wandb table with 

> 1. ouriginal sound string
> 2. playable sound file
> 3. 4 * 1 second clips files
> 4  4 * 1 second clips files

# Sound and Spectrogram Table



## Adding Sound and Spectrograms to WandB Table  🔉 🪄 🐝

In [79]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
n_examples = 2
n_samples = 10
s_df = df.sample(n=n_samples, random_state=2)
columns = list(df.columns)
table_with_media = wandb.Table(data=s_df,columns=columns)
for colmn in df.columns[-n_examples:]:
    # create wandb sound objects
    sounds = [wandb.Audio(fid) for fid in tqdm(s_df[colmn].values,**tqdm_args,desc=colmn)]
    # add these as a column to our wandb table
    table_with_media.add_column(name=f'sound_{colmn}',data=sounds)

5_6_sec: 100%|████████████████████████████████| 10/10 [00:00<00:00, 3980.93it/s]


### Adding Spectrograms to wandb Table

In [80]:
for col in df.columns[-n_examples:]:
    # read wavs 
    wavs = map(read_wav,tqdm(s_df[col].values,**tqdm_args,desc=col))
    # generate spectrograms
    spects = map(get_arm_spectrogram,wavs)
    sample_rates = iter(np.full((1,len(df)),1600)[0])
    # get numpy arrays of save .jpg files
    ims = list(map(plot_spectrogram,spects,sample_rates))
    # create a list of wandb images
    spects = list(map(wandb.Image,tqdm(ims)))
    # add these to a table 
    table_with_media.add_column(name=f'image_{col}',data=spects)
run.log({'sound_spectrogram_table':table_with_media})

100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]


## Stack our pandas df by 1 second Files 🧱

In [81]:
rp_df = pd.DataFrame(np.repeat(df.values, 6, axis=0))
rp_df.columns=df.columns
a,b,c,d,e,f = np.array([df[col].values for col in df.columns[-6:]])
all_files =np.vstack((a,b,c,d,e,f)).reshape((-1,),order='F')
rp_df = rp_df[df.columns[:-6]]
rp_df['all_files']=all_files
df = rp_df
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


### Define Test,Train, Val using pandas & add sets column to our dataset

In [82]:
sets = { }
train_val = df.sample(frac=0.9,random_state=42)
sets['test']=df.drop(train_val.index)
sets['train']=train_val.sample(frac=0.9,random_state=42)
sets['val']=train_val.drop(sets['train'].index)
for name,subset in sets.items():
        subset = subset.reset_index(drop=True)
        subset['set']=[name for va in range(len(subset))]
        sets[name]=subset
df.groupby("target", group_keys=False).apply(lambda x: x)
df = pd.concat(sets.values()).reset_index(drop=True)
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [91]:
df

,Unnamed: 0.1,Unnamed: 0,filename,fold,target,category,esc10,src_file,take,all_files,set,is_sound
0,3,3,1-103995-A-30.wav,1,30,door_wood_knock,False,103995,A,ESC-50/door_wood_knock/3_1-103995-A-30.wav,test,True
1,4,4,1-103999-A-30.wav,1,30,door_wood_knock,False,103999,A,ESC-50/door_wood_knock/4_1-103999-A-30.wav,test,True
2,9,9,1-115546-A-48.wav,1,48,fireworks,False,115546,A,ESC-50/fireworks/1_1-115546-A-48.wav,test,True
3,10,10,1-115920-A-22.wav,1,22,clapping,False,115920,A,ESC-50/clapping/0_1-115920-A-22.wav,test,True
4,11,11,1-115920-B-22.wav,1,22,clapping,False,115920,B,ESC-50/clapping/0_1-115920-B-22.wav,test,True
...,...,...,...,...,...,...,...,...,...,...,...,...
71995,7488,7488,4-204115-A-39.wav,4,39,glass_breaking,False,204115,A,ESC-50/glass_breaking/4_4-204115-A-39.wav,val,True
71996,9642,9642,2-118072-A-0.wav,2,0,dog,True,118072,A,ESC-50/dog/5_2-118072-A-0.wav,val,True
71997,6280,6280,4-189836-A-22.wav,4,22,clapping,False,189836,A,ESC-50/clapping/4_4-189836-A-22.wav,val,True
71998,1177,1177,5-252248-A-34.wav,5,34,can_opening,False,252248,A,ESC-50/can_opening/3_5-252248-A-34.wav,val,False


# remove silence by randomly sampling from non silent

In [83]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
is_sound = [not np.std(read_wav(x)) < 0.1 for x in df.all_files.values]
df['is_sound']=is_sound
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [84]:
silence = df[df.is_sound==False]
final_meta_table = wandb.Table(data=silence,columns=list(df.columns))
wandb_audio = list(map(wandb.Audio,silence.all_files.values))
final_meta_table.add_column(name='audio',data=wandb_audio)
run.log({'silence_tabel':final_meta_table})

In [85]:
run

# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [86]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


In [87]:
for pth in df[df.is_sound==False].all_files.values:
    path = Path(pth)
    try:
        path.unlink()
    except FileNotFoundError:
        print('alredy deleted')
for path in tqdm(esc_artifacts,**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

100%|███████████████████████████████████████████| 51/51 [00:15<00:00,  3.34it/s]


## This Function is for Data Augmentation 🎛️

In [95]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift,RoomSimulator,ClippingDistortion,AddBackgroundNoise
def augmenter(audio,sample_rate):
    audio = np.array([audio]).astype(np.float32)[0]
    augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),
    TimeStretch(min_rate=0.8, max_rate=0.9, p=0.1),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.3),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.3),
    RoomSimulator(p=0.5),
    ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=10, p = 0.1),
    AddBackgroundNoise(sounds_path='test.wav',p=0.5)
    ])
    return augment(samples=audio, sample_rate=sample_rate).astype(np.int16)[:44100]

ImportError: cannot import name 'log_wandb_artifact' from 'utils' (/Users/fridadesigley/pico/tiny-ml/utils/__init__.py)

AttributeError: module 'utils.data_processing' has no attribute 'augmenter'

# Save 3D arrays of Spectrograms using numpy

In [89]:
def save_npz(x,y,name,augmenter=None):
    wavs = list(map(read_wav,tqdm(x,f'creating {name}')))
    if augmenter:
        samples = np.full((1,len(y)), 16000)[0]
        wavs = list(map(augmenter,wavs,tqdm(samples,f'augmenting {name}')))
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} augspects'))))
    else:
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} spects'))))
    for array in x_data:
        assert array.shape==(682, 257)
    np.savez(f'{name}.npz',x_data=x_data,y_data=y)

## Loop over files and save as npz, this is our pre-processed training data.

In [93]:
contains_sound = df[df.is_sound==True]
subset =  contains_sound[contains_sound.set=='train']


In [ ]:
pre_process=True
contains_sound = df[df.is_sound==True]
if pre_process:
    for name in df.set.unique():
        subset =  contains_sound[contains_sound.set==name]
        x_data = subset['all_files'].values
        y_data = subset['target'].values
        save_npz(x_data,y_data,f'processed/{name}')
        if name=='train':
            print('augmenting')
            save_npz(x_data[:len(x_data)//5],y_data[:len(y_data)//5],f'processed/{name}_aug',augmenter=augmenter)

creating processed/train spects: 100%|███| 55257/55257 [07:23<00:00, 124.57it/s]


# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [174]:
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)
run.finish()

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


In [ ]:
run

### Training load training dataset which has already been pre processed

In [46]:
with np.load('processed/train.npz',allow_pickle=True) as data:
    train_x = data['x_data']
    train_y = data['y_data'].astype(np.uint8)
with np.load('processed/train_aug.npz',allow_pickle=True) as data:
    aug_x = data['x_data']
    aug_y = data['y_data'].astype(np.uint8)
with np.load('processed/test.npz',allow_pickle=True) as data:
    test_x = data['x_data']
    test_y = data['y_data'].astype(np.uint8)
with np.load('processed/val.npz',allow_pickle=True) as data:
    val_x = data['x_data']
    val_y = data['y_data'].astype(np.uint8)
    


In [47]:
type(train_x)

numpy.ndarray

## create TensoFlow Dataset from numpy arrays of spectrograms and target variable

In [64]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_aug_dataset = tf.data.Dataset.from_tensor_slices((aug_x, aug_y))
train_dataset = train_dataset.concatenate(train_aug_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y))

train_ds = train_dataset.cache().shuffle(10000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [65]:
next(iter(train_ds))

2023-01-29 19:02:05.515235: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(<tf.Tensor: shape=(32, 682, 257), dtype=float32, numpy=
 array([[[6.1296875e+01, 3.3656250e+01, 3.5937500e+00, ...,
          1.0625000e+00, 1.5468750e+00, 3.4375000e-01],
         [6.3812500e+01, 3.7500000e+01, 7.8437500e+00, ...,
          1.0468750e+00, 1.4843750e+00, 7.3437500e-01],
         [5.9984375e+01, 3.6312500e+01, 1.4375000e+01, ...,
          1.5937500e+00, 1.5312500e+00, 3.9062500e-01],
         ...,
         [3.9843750e+01, 3.8781250e+01, 1.5562500e+01, ...,
          7.9687500e-01, 7.6562500e-01, 4.6875000e-01],
         [2.5906250e+01, 3.3734375e+01, 2.6218750e+01, ...,
          1.0937500e+00, 1.3750000e+00, 1.1093750e+00],
         [1.4640625e+01, 1.8078125e+01, 4.2578125e+01, ...,
          1.4531250e+00, 1.7656250e+00, 1.4843750e+00]],
 
        [[9.4171875e+01, 5.3296875e+01, 9.0781250e+00, ...,
          2.4531250e+00, 1.3437500e+00, 4.0625000e-01],
         [8.2062500e+01, 4.4734375e+01, 1.2875000e+01, ...,
          2.3437500e+00, 1.2500000e+00, 3.5937500e-01]

## Test that this dataset is loading okay

In [67]:
x,y = next(iter(train_dataset.take(1)))
print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

 target = 10, 
 spectrogram = 
 [[14.6875   10.984375  8.125    ...  2.484375  2.765625  4.21875 ]
 [ 7.09375  10.1875   10.484375 ...  2.140625  1.921875  4.453125]
 [ 0.609375  9.265625 14.96875  ...  1.671875  1.859375  4.484375]
 ...
 [ 2.265625  4.453125  3.75     ...  5.6875    4.59375   0.40625 ]
 [ 1.1875    5.765625  6.4375   ...  5.3125    4.03125   2.484375]
 [ 4.453125  9.6875    7.171875 ...  3.203125  3.375     2.734375]]
(682, 257, 1)


In [68]:
print(x)

tf.Tensor(
[[14.6875   10.984375  8.125    ...  2.484375  2.765625  4.21875 ]
 [ 7.09375  10.1875   10.484375 ...  2.140625  1.921875  4.453125]
 [ 0.609375  9.265625 14.96875  ...  1.671875  1.859375  4.484375]
 ...
 [ 2.265625  4.453125  3.75     ...  5.6875    4.59375   0.40625 ]
 [ 1.1875    5.765625  6.4375   ...  5.3125    4.03125   2.484375]
 [ 4.453125  9.6875    7.171875 ...  3.203125  3.375     2.734375]], shape=(682, 257), dtype=float32)


In [69]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_dataset.map(lambda x, y: tf.reshape(x, input_shape)))

# we have made an application and embeded wandb in the recording process
this is to allow us to :
> - capture new data from you device 🔊
> - Version data using wandb Atefact 🏺
> - Define your wakework 📢

In [71]:
!tree data -d


data
├── background
├── no
└── yes

3 directories


# Create a new training run for our training 🏃🏻‍♀️ ♀️ 

In [73]:
run = wandb.init(config={"bs": 12},entity='tiny-ml',project = 'wake_word_detection', group='training')

epoch/accuracy,▁▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
epoch/learning_rate,██████████▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,██████████▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
epoch/val_accuracy,▁▆▇▇▇▇▇▆▇▆▆▆▇▇▇█▇▇███▇███▇▇██▇██
epoch/val_loss,█▄▃▃▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁
epoch/accuracy,0.33651
epoch/epoch,31
epoch/learning_rate,0.00033
epoch/loss,2.44278


## Define our model 🏗️

In [75]:
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

In [76]:
baseline_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_1 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 288)               0         
                                                                 
 dropout_1 (Dropout)         (None, 288)              

In [77]:
METRICS = ["accuracy",]
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=METRICS,
)

In [78]:
def scheduler(epoch, lr):
    ''' a function to increase lr at start of trining
    '''
    if epoch < 10:
        return lr
    else:
        # add somthing like np.linespace([0,-0.1])
        return lr * tf.math.exp(-0.1)

In [79]:
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
# Initialize a new W&B run
checkpoint_path = "training_1/"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights

In [80]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [81]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

In [84]:
run

In [83]:
EPOCHS = 250
history = baseline_model.fit(
     train_ds, 
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_ds
)

Epoch 1/250
358/365 [============================>.] - ETA: 0s - loss: 3.5079 - accuracy: 0.1027
Epoch 1: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


365/365 [==============================] - 3s 7ms/step - loss: 3.5046 - accuracy: 0.1035 - val_loss: 3.1314 - val_accuracy: 0.1713 - lr: 0.0030
Epoch 2/250
364/365 [============================>.] - ETA: 0s - loss: 3.1033 - accuracy: 0.1947
Epoch 2: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


...

INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


365/365 [==============================] - 2s 6ms/step - loss: 2.3822 - accuracy: 0.3525 - val_loss: 2.8045 - val_accuracy: 0.2759 - lr: 9.0827e-06
Epoch 69/250
356/365 [============================>.] - ETA: 0s - loss: 2.3728 - accuracy: 0.3515
Epoch 69: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


365/365 [==============================] - 2s 6ms/step - loss: 2.3741 - accuracy: 0.3513 - val_loss: 2.8046 - val_accuracy: 0.2759 - lr: 8.2183e-06


In [ ]:
run.finish()

### Evaluate our locally saved model

In [86]:
latest = tf.train.latest_checkpoint('training_1')
baseline_model.load_weights(latest)

2023-01-29 19:06:39.183283: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open training_1/.: FAILED_PRECONDITION: training_1; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


### Evaluate our Pre-trained Model

In [88]:
baseline_model.evaluate(test_ds, verbose=2)

214/214 - 2s - loss: 2.3728 - accuracy: 0.3750 - 2s/epoch - 8ms/step


[2.3727927207946777, 0.3750365674495697]

### Dowload model artifact/ our registered model  

In [91]:
api = wandb.Api()
artifact = api.artifact('tiny-ml/wake_word_detection/run_xt4lgm8s_model:latest', type='model')
print(artifact.digest,artifact.aliases)
artifact.download()  

7818bd64cbe173268a53d74e25cb04f1 ['latest', 'epoch_68']


wandb:   15 of 15 files downloaded.  


'./artifacts/run_xt4lgm8s_model:v68'

### Load our pre trained tiny model from wandb artifact

In [92]:
baseline_model.load_weights('./artifacts/run_xt4lgm8s_model:v68/cp.ckpt')
baseline_model.evaluate(test_ds, verbose=2)

214/214 - 2s - loss: 2.5362 - accuracy: 0.3197 - 2s/epoch - 8ms/step


[2.5361759662628174, 0.3197249174118042]

In [93]:
path = Path('./data/') 
paths = {pth.name:list(map(str,filter(lambda x:x.name != '.gitkeep',pth.iterdir()))) 
         for pth in path.iterdir() if pth.name in ['background','yes']}
paths = {k:np.stack(list(filter(lambda x:len(x)==16384,map(read_wav,v)))) 
         for (k,v) in paths.items()}

for k,v in paths.items():
    filler_x = [augmenter(x,sample_rate) 
                for x in v[np.random.choice(v.shape[0], 500-len(v), replace=True)]]
    filler_x = np.stack(list(map(lambda x:x[:16384], filler_x)))
    paths[k]=np.array(list(map(get_arm_spectrogram,np.vstack([v,filler_x]))))


NameError: name 'augmenter' is not defined

In [94]:
paths.values()

dict_values([array([[   0,    0,    0, ..., -227, -295, -311],
       [   0,    0,    0, ..., -327, -279, -227],
       [   0,    0,    0, ..., -391, -395, -443],
       ...,
       [   0,    0,    0, ..., -483, -535, -499],
       [   0,    0,    0, ..., -239, -267, -303],
       [   0,    0,    0, ..., -439, -403, -387]], dtype=int16), array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ..., -107, -123, -115],
       [   0,    0,    0, ..., -275, -275,  -83],
       ...,
       [   0,    0,    0, ..., -347, -363, -447],
       [   0,    0,    0, ..., -619, -603, -603],
       [   0,    0,    0, ...,  -59,  -63,  -51]], dtype=int16)])

In [ ]:
y = np.concatenate([np.zeros((1,500)).astype(np.uint16)[0], np.ones((1,500)).astype(np.uint16)[0]])
x = np.vstack(list(paths.values()))

In [ ]:
train_wake_data = tf.data.Dataset.from_tensor_slices((x, y))
train_wake_loader = train_wake_data.cache().shuffle(2000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
next(iter(train_wake_loader.take(1)))

In [ ]:
x,y = next(iter(train_wake_data.take(1)))
print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_wake_data.map(lambda x, y: tf.reshape(x, input_shape)))


In [ ]:
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

In [ ]:
base_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

In [ ]:
wake_word_model.load_weights('./artifacts/run_ao5qr95h_model:v26/cp.ckpt')

In [ ]:
model_body = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
classifier_head = tf.keras.layers.Dense(1, activation="sigmoid")(model_body.output)
wake_word_model = tf.keras.Model(model_body.input, classifier_head)
wake_word_model.summary()

In [ ]:
for layer in wake_word_model.layers:
    layer.trainable = False
wake_word_model.layers[-1].trainable = True

In [ ]:
# Initialize a new W&B run
checkpoint_path = "training_2/"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

In [ ]:
wake_word_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS,
)

In [24]:
run = wandb.init(config={"bs": 12},
                 entity='tiny-ml',
                 project='wake_word_detection', group='training',
                settings={'quiet':True,'silent':True,'show_warnings':False,'show_info':False})
run

In [ ]:
EPOCHS = 100
history = wake_word_model.fit(
    train_wake_loader, 
    epochs=EPOCHS,
    callbacks=callbacks
)